In [ ]:
import pandas as pd
import plotly.express as px
import requests
import json
import datetime
import concurrent.futures
import swifter
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from numpy import random
import math
import seaborn as sns
plt.style.use('fivethirtyeight')
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')
import dateutil

In [ ]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

In [ ]:
if os.path.isfile('dados/TRF4_JE_2018-01-01.csv'):
    df_tribunal = pd.read_csv('dados/TRF4_JE_2018-01-01.csv', header='infer', sep=';', compression='zip')
else:
    df_tribunal = pd.read_csv(glob.glob("dados/TJRN_17*.csv")[0], header='infer', sep=';', compression='zip')
    print(f'Carregando o dataset referente ao {glob.glob("dados/TJRN_17*.csv")[0]}')
    

In [ ]:
df_tribunal['codigo_classe'].value_counts()

In [ ]:
df_tribunal.drop_duplicates(inplace=True)
if 'sort' in df_tribunal.columns:
    df_tribunal.drop(columns=['sort'], inplace=True)
if 'codigo' in df_tribunal.columns:
    df_tribunal.drop(columns=['codigo'], inplace=True)    
if 'classe' in df_tribunal.columns:
    df_tribunal.drop(columns=['classe'], inplace=True)    
if 'numero_processo' in df_tribunal.columns:
    df_tribunal.drop(columns=['numero_processo'], inplace=True)    
if 'orgao_julgador' in df_tribunal.columns:
    df_tribunal.drop(columns=['orgao_julgador'], inplace=True)
    
if df_tribunal['assuntos'].isna().sum() > (0.8 * len(df_tribunal['assuntos'])):
    df_tribunal.drop(columns=['assuntos'], inplace=True)  
    


In [ ]:
df_tribunal.head(1)

In [ ]:
try:
    df_tribunal['data_ajuizamento'] = pd.to_datetime(df_tribunal['data_ajuizamento'], errors='coerce')
except pd.errors.OutOfBoundsDatetime:
    df_tribunal['data_ajuizamento'] = df_tribunal[df_tribunal.astype(bool)]
    df_tribunal['data_ajuizamento'] = df_tribunal.dropna()
    df_tribunal['data_ajuizamento'] = df_tribunal.apply(dateutil.parser.parse)

try:
    df_tribunal['ultima_atualizacao'] = pd.to_datetime(df_tribunal['ultima_atualizacao'], errors='coerce')
except pd.errors.OutOfBoundsDatetime:
    df_tribunal['ultima_atualizacao'] = df_tribunal[df_tribunal.astype(bool)]
    df_tribunal['ultima_atualizacao'] = df_tribunal.dropna()
    df_tribunal['ultima_atualizacao'] = df_tribunal.apply(dateutil.parser.parse)
#print(df_tribunal.info())
#df_tribunal['data_ajuizamento'] = df_tribunal['data_ajuizamento'].dt.strftime('%Y-%m-%d')
#df_tribunal['ultima_atualizacao'] = pd.to_datetime(df_tribunal['ultima_atualizacao'], format='mixed').dt.strftime('%Y-%m-%d')
#print(df_tribunal.head(2))

df_tribunal.dropna(inplace=True)

In [ ]:
movimentacoes = list(df_tribunal.movimentos.values)
print(len(movimentacoes))

In [ ]:
lista_movimentacoes_completa = []
lista_codigos = list()
lista_nomes = list()
lista_data = list()
i = 0

for movimento in movimentacoes:
    if '272' in movimento.split():
        print(movimento)
        #continue 
        #print(len(list(movimento.split(',')))/4)
    else:
        if len(list(movimento.split(','))) % 4 == 0:
            lista_temp = list()
            for elemento in range(0,int(len(list(movimento.split(',')))/4),4):
                codigo = movimento.split(',')[elemento].replace('[','').replace(' ', '')
                nome_movimentacao = movimento.split(',')[elemento+1].replace('[','').replace(' ', '')
                data_movimentacao = movimento.split(',')[elemento+2].replace('[','').replace('Timestamp(','').split(' ')[1].replace(' ', '').replace('\'', '')
                lista_temp.append([codigo, nome_movimentacao, data_movimentacao])
        else:
            lista_temp.append([-1,-1,-1])
    lista_movimentacoes_completa.append(lista_temp)


In [ ]:
print(type(lista_movimentacoes_completa), len(lista_movimentacoes_completa))
print(type(lista_movimentacoes_completa[0]), len(lista_movimentacoes_completa[2]))
print(lista_movimentacoes_completa[5200])
print(lista_movimentacoes_completa[:10])

In [ ]:
tribunal = 'TRF'
if tribunal[:2] == 'TJ':
    movimentacoes_gabinete = pd.read_csv('dados/movimentos_gabinete.csv', sep=';', header='infer')
    movimentacoes_gabinete['codigo'] = movimentacoes_gabinete['vazio.1']
    #movimentacoes_gabinete.drop(columns=['complemento','vazio', 'vazio.2', 'vazio.1'], inplace=True)
    lista_movimentacoes_gabinete = movimentacoes_gabinete['Código'].to_list()
    #lista_movimentacoes_sentenca = movimentacoes_gabinete[movimentacoes_gabinete['tipo'] == 'Julgamento']['Código'].to_list()
    movimentacoes_secretaria = pd.read_csv('dados/movimentos_secretaria.csv', sep=';', header='infer')
    #movimentacoes_secretaria.drop(columns=['Unnamed: 3','complemento'], inplace=True)
    lista_movimentacoes_secretaria = movimentacoes_secretaria['codigo'].to_list()
else:
    movimentacoes_gabinete = pd.read_csv('dados/jf_movimentos_gabinete.csv', sep=';', header='infer')
    print(movimentacoes_gabinete.head(2))
    #movimentacoes_gabinete['codigo'] = movimentacoes_gabinete['vazio.1']
    #movimentacoes_gabinete.drop(columns=['complemento','vazio', 'vazio.2', 'vazio.1'], inplace=True)
    lista_movimentacoes_gabinete = movimentacoes_gabinete['codigo'].to_list()
    #lista_movimentacoes_sentenca = movimentacoes_gabinete[movimentacoes_gabinete['tipo'] == 'Julgamento']['codigo'].to_list() 
    movimentacoes_secretaria = pd.read_csv('dados/jf_movimentos_secretaria.csv', sep=';', header='infer')
    print(movimentacoes_secretaria.head(2))
    #movimentacoes_secretaria.drop(columns=['Unnamed: 3','complemento'], inplace=True)
    lista_movimentacoes_secretaria = movimentacoes_secretaria['codigo'].to_list()           
#print(movimentacoes_gabinete.head(5))

In [ ]:
#print(movimentacoes_gabinete[movimentacoes_gabinete['nome'] == 'Julgamento'].head(5))

In [ ]:

#print(movimentacoes_secretaria.head(5))

#print(movimentacoes_secretaria.head(5))

In [ ]:
def calcular_data_sentenca(movimentacoes):
    processos_sentenciados = 0
    processos_nao_sentenciados = 0    
    lista_data_sentenca = list()
    for i in range(0,(len(movimentacoes)),1):
        processado = False
        if len(movimentacoes[i]) > 3:
            for j in range(0,(len(movimentacoes[i])-1),3):
                #print(movimentacoes[i][j][0])
                if processado:
                   break 
                if int(movimentacoes[i][j][0]) in lista_movimentacoes_gabinete:
                    data = pd.to_datetime(movimentacoes[i][j][2])
                    data = data.date().strftime("%Y-%m-%d")
                    lista_data_sentenca.append(data)
                    processos_sentenciados = processos_sentenciados + 1
                    processado = True
                    #print(f'Processando: {i}: Sentença encontrada')
            if not processado:
                lista_data_sentenca.append('-1')
                processado = True
                processos_nao_sentenciados = processos_nao_sentenciados + 1         
   
                #print(f'data adicionada: {data}')    
        else:
            #print(movimentacoes[i])
            lista_data_sentenca.append('-1')
            processos_nao_sentenciados = processos_nao_sentenciados + 1
            #print(f'data adicionada: {data_default_nao_sentenciados}')
            #print(f'Processando: {i}: SEM movimentação do gabinete')
    print(f'processos sentenciados: {processos_sentenciados}\tprocessos não sentenciados: {processos_nao_sentenciados}')
    return lista_data_sentenca

In [ ]:
#print(calcular_data_sentenca(lista_movimentacoes_completa))
print(len(lista_movimentacoes_completa), len(calcular_data_sentenca(lista_movimentacoes_completa)))

In [ ]:
# funciona ok.
def calcular_tempo_entre_movimentacoes(movimentacoes, mov_inicial, mov_final):
    lista_duracao = list()
    for i in range(0,(len(movimentacoes)),1):
        #if i < 3:
            #print(movimentacoes[i])
        if ((len(movimentacoes[i]) > 3) and (len(movimentacoes[i]) >= mov_final)):
            try:
                duracao = abs(pd.to_datetime(movimentacoes[i][mov_final][2]) - pd.to_datetime(movimentacoes[i][mov_inicial][2]))/np.timedelta64(1, 'D')
                lista_duracao.append(int(duracao))
            except:
                lista_duracao.append(int(-1))          

        else: 
            lista_duracao.append(int(-1))
            
    return lista_duracao        

In [ ]:
df_tribunal['data_sentenca'] = calcular_data_sentenca(lista_movimentacoes_completa)
df_tribunal['movimentos'] = lista_movimentacoes_completa

In [ ]:
def estatisticas_movimentacoes_processo(movimentacoes):
    min = np.inf
    max = 0
    media = 0.0
    lista_mov_processos = list()
    for movimento in movimentacoes:
        lista_mov_processos.append(len(movimento)/3)
        if len(movimento)/3 > max:
            max = len(movimento)/3
        elif len(movimento)/3 < min:
            min = len(movimento)/3
        media += len(movimento)/3
    media = media / len(movimentacoes)
    return media, lista_mov_processos

In [ ]:
df_tribunal['tempo_entre_1e2_mov'] = calcular_tempo_entre_movimentacoes(df_tribunal['movimentos'].to_list(),0,1)
df_tribunal['tempo_entre_2e3_mov'] = calcular_tempo_entre_movimentacoes(df_tribunal['movimentos'].to_list(),1,2)
df_tribunal['tempo_entre_3e4_mov'] = calcular_tempo_entre_movimentacoes(df_tribunal['movimentos'].to_list(),2,3)
#df_tribunal['tempo_entre_4e5_mov'] = calcular_tempo_entre_movimentacoes(df_tribunal['movimentos'].to_list(),4,5)

df_tribunal['mais60d'] = (df_tribunal['tempo_entre_1e2_mov'] > 60) | (df_tribunal['tempo_entre_2e3_mov'] > 60) | (df_tribunal['tempo_entre_3e4_mov'] > 60) #| (df_tribunal['tempo_entre_4e5_mov'] > 60)
df_tribunal['mais60d'].replace(['False','True'],[0,1],inplace=True)

In [ ]:
print(df_tribunal['mais60d'].value_counts())

In [ ]:
df_sentenciados = df_tribunal[df_tribunal['data_sentenca'] != '-1']
df_sentenciados['tempo_ate_sentenca'] = abs(pd.to_datetime(df_sentenciados['data_sentenca'], utc=True) - pd.to_datetime(df_sentenciados['data_ajuizamento'], utc=True))/np.timedelta64(1, 'D')
df_nao_sentenciados = df_tribunal[df_tribunal['data_sentenca'] == '-1']
df_nao_sentenciados['tempo_ate_sentenca'] = 0

In [ ]:
print(df_sentenciados.shape)
print(df_nao_sentenciados.shape)
print(df_tribunal.shape)

In [ ]:
df_tribunal = pd.concat([df_sentenciados, df_nao_sentenciados])
#print(df_tribunal.head(1))

In [ ]:
tribunal = df_tribunal['tribunal'][0]
grau = df_tribunal['grau'][0]
data_inicio = df_tribunal['data_ajuizamento'].min().date().strftime("%Y-%m-%d")
data_final = df_tribunal['data_ajuizamento'].max().date().strftime("%Y-%m-%d")
classe = df_tribunal['codigo_classe'][0]
nome_dataset = tribunal + '_' + grau + '_' + str(classe) + '_' + str(data_inicio) + '-' + str(data_final)
df_tribunal.to_csv(f'dados/processados/{nome_dataset}.csv', sep=';', header=True, index=False, compression='zip')
nome_dataset = tribunal + '_' + grau + '_' + str(classe) + '_' + str(data_inicio) + '-' + str(data_final) + '_' + 'sentenciados'
df_sentenciados.to_csv(f'dados/processados/{nome_dataset}.csv', sep=';', header=True, index=False, compression='zip')
nome_dataset = tribunal + '_' + str(classe) + '_' + str(data_inicio) + '-' + str(data_final) + '_' + 'nao_sentenciados'
df_nao_sentenciados.to_csv(f'dados/processados/{nome_dataset}.csv', sep=';', header=True, index=False, compression='zip')

## EDA

In [ ]:
df_tribunal.isna().sum()

In [ ]:
df_tribunal.info()

In [ ]:
#print(df_sentenciados.shape[0], df_nao_sentenciados.shape[0])
percentual_sentenciados = (int(df_sentenciados.shape[0])/int(df_nao_sentenciados.shape[0]) * 100)
percentual_sentenciados = float("{:.4f}".format(percentual_sentenciados))
print(f'Percentual da base de casos sentenciados: {percentual_sentenciados}%')

In [ ]:
fig = px.histogram(df_sentenciados, x='data_ajuizamento', marginal='rug', title='Distribuição dos processos já sentenciados pela data de ajuizamento')
fig.show()

In [ ]:
fig = px.histogram(df_nao_sentenciados, x='data_ajuizamento', marginal='rug', title='Distribuição dos processos ainda não sentenciados pela data de ajuizamento')
fig.show()

In [ ]:
fig = px.histogram(df_sentenciados, x='tempo_ate_sentenca', marginal='rug', title='Distribuição do tempo até a sentença')
fig.show()

In [ ]:
media, lista_mov_processos = estatisticas_movimentacoes_processo(df_sentenciados['movimentos'].to_list())
print(f'Média de movimentações por processo: {media}')
fig = px.histogram(lista_mov_processos, marginal='rug', title='Distribuição do número de movimentações por processo',
                   nbins=math.ceil((media *2)), labels={'x':'Movimentações', 'y':'Ocorrências'}).update_layout(
    yaxis_title="Ocorrências", xaxis_title="Movimentações")
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(y=df_sentenciados['tempo_entre_2e3_mov'], 
                     name='2ª e 3ª movimentação', marker_color = 'indianred'))
fig.add_trace(go.Box(y=df_sentenciados['tempo_entre_3e4_mov'], 
                     name='3ª e 4ª movimentação', marker_color = 'lightseagreen'))
fig.update_layout(title='Distribuição do tempo necessário para realizar as duas primeiras movimentações processuais')
fig.show()


In [ ]:
#df_sentenciados.groupby(['data_sentenca'])['tempo_ate_sentenca'].count()
df_ordenado = df_sentenciados.sort_values(by='data_sentenca')
fig = px.line(df_ordenado, x='data_sentenca', y='tempo_ate_sentenca', title='Tempo até sentença ao longo dos anos').update_layout(
    yaxis_title="Tempo até sentença", xaxis_title="Ano")
fig.show()

In [ ]:
top99_sentenciados = df_sentenciados['tempo_ate_sentenca'].quantile(0.80)
print(f'(80% dos processos são sentenciados em até {top99_sentenciados:.1f} dias)')

In [ ]:
sns.set()
fig = sns.pairplot(df_sentenciados[['tempo_entre_1e2_mov', 'tempo_entre_2e3_mov', 
                             'tempo_entre_3e4_mov', 'mais60d', 'tempo_ate_sentenca']], size = 2.5)
fig.suptitle('Relacionamento entre as variáveis', y=1.03)
plt.show()

In [ ]:
sns.heatmap(df_sentenciados[['tempo_entre_1e2_mov', 'tempo_entre_2e3_mov', 
                             'tempo_entre_3e4_mov', 'mais60d', 'tempo_ate_sentenca']].corr(),annot=True,cmap='RdYlGn',linewidths=0.2)
fig=plt.gcf()
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

In [ ]:
def lista_movimentacoes(movimentacoes, posicao):
    movimentacoes_posicao_3 = list()
    for movimento in movimentacoes:
        movimentacoes_posicao_3.append(movimento[posicao -1][0])
    return list(set(movimentacoes_posicao_3))

    
mais_demoradas_movimentacoes_pos_3 = lista_movimentacoes(df_sentenciados['movimentos'].to_list(), 3)
print(mais_demoradas_movimentacoes_pos_3)


for codigo in mais_demoradas_movimentacoes_pos_3:
    acoes_secretaria = 0
    acoes_gabinete = 0
    acoes_sentenca = 0
    if int(codigo) in lista_movimentacoes_secretaria:
        acoes_secretaria += 1
    elif int(codigo) in lista_movimentacoes_gabinete:
        acoes_gabinete += 1
    elif int(codigo) in lista_movimentacoes_gabinete:
        acoes_sentenca += 1

print(f'Das movimentações mais demoradas, {acoes_secretaria} são realizadas pela secretaria, {acoes_gabinete} pelo gabinete e {acoes_sentenca} são sentenças')
        

## Testes com ML

In [ ]:
print(df_tribunal.columns)

In [ ]:
df_ml = df_sentenciados.drop(columns=['movimentos', 'numero_processo', 'data_ajuizamento', 'ultima_atualizacao', 'codigo', 'data_sentenca' ])
df_ml = pd.get_dummies(df_ml, columns=['assuntos'], drop_first=True)
df_x = df_ml.drop(columns=['tempo_ate_sentenca', 'mais60d', 'tempo_entre_3e4_mov'])
#print(df_ml.head(1))
print(df_ml.columns)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold

metricas = ['accuracy', 'f1', 'precision', 'recall']
rfc = RandomForestClassifier(random_state=19, n_jobs=-1)
adc = AdaBoostClassifier(random_state=19,algorithm="SAMME", n_estimators=200)
mlp = MLPClassifier(random_state=19)
dtc = DecisionTreeClassifier(random_state=19)

scores_rfc = cross_validate(rfc, df_x,
                             df_ml['mais60d'], cv=10, scoring=metricas)
scores_adc = cross_validate(adc, df_x,
                             df_ml['mais60d'], cv=10, scoring=metricas)
scores_mlp = cross_validate(mlp, df_x,
                             df_ml['mais60d'], cv=10, scoring=metricas)
scores_dtc = cross_validate(dtc, df_x,
                             df_ml['mais60d'], cv=10, scoring=metricas)

In [ ]:
chaves = scores_dtc.keys()
print(f'Usando as colunas {df_x.columns} chegou-se ao seguinte resultado\n\n')
print(f'Métrica \tRandomForest \tAdaBoost \tMLP \tDecisionTree')
for chave in chaves:
    print(f'{chave}:\t {scores_rfc[chave].mean():.3f} \t\t{scores_adc[chave].mean():.3f} \t\t{scores_mlp[chave].mean():.3f} \t\t{scores_dtc[chave].mean():.3f}')


### Modelo tentando prever se vai atrasar ou não

In [ ]:
from sklearn.tree import export_text
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_x, df_ml['mais60d'],test_size=0.20, random_state=19)

clf = DecisionTreeClassifier(max_depth=10).fit(X_train, y_train)
print(clf.score(X_test,y_test))
r = export_text(clf, feature_names=df_x.columns)

print(r)
 

### Regressão para o tempo até a sentença

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_x, 
                                                    df_ml['tempo_ate_sentenca'], test_size=0.20, random_state=19)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


model = RandomForestRegressor(max_depth=50, random_state=0, n_estimators=250)
model.fit(X_train, y_train)
rf_y_pred = model.predict(X_test)

rf_r2 = r2_score(y_test, rf_y_pred)
rf_mae = mean_absolute_error(y_test, rf_y_pred)
rf_rmse = mean_squared_error(y_test, rf_y_pred, squared=False)
rf_mse = mean_squared_error(y_test, rf_y_pred)

print('----------------------------------------------------')

print(f'AdaBoost Regressor')
regr = AdaBoostRegressor(random_state=0, n_estimators=100)
regr.fit(X_train, y_train)
ada_y_pred = regr.predict(X_test)

ada_r2 = r2_score(y_test, ada_y_pred)
ada_mae = mean_absolute_error(y_test, ada_y_pred)
ada_rmse = mean_squared_error(y_test, ada_y_pred, squared=False)
ada_mse = mean_squared_error(y_test, ada_y_pred)

print('----------------------------------------------------')

print(f'MLP Regressor')

regr_mlp = MLPRegressor(random_state=1, max_iter=500).fit(X_train, y_train)
mlp_y_pred = regr_mlp.predict(X_test)

mlp_r2 = r2_score(y_test, mlp_y_pred)
mlp_mae = mean_absolute_error(y_test, mlp_y_pred)
mlp_rmse = mean_squared_error(y_test, mlp_y_pred, squared=False)
mlp_mse = mean_squared_error(y_test, mlp_y_pred)

print('----------------------------------------------------')

print(f'DT Regressor')
regressor = DecisionTreeRegressor(random_state=0).fit(X_train, y_train)
dt_y_pred = regr_mlp.predict(X_test)

dt_r2 = r2_score(y_test, dt_y_pred)
dt_mae = mean_absolute_error(y_test, dt_y_pred)
dt_rmse = mean_squared_error(y_test, dt_y_pred, squared=False)
dt_mse = mean_squared_error(y_test, dt_y_pred)

print(f'Metric\t\t\t\t RF\t AdaBoost\t MLP\t\t DT')
print(f"Mean Squared Error:\t\t {rf_r2:.3f}\t {ada_r2:.3f}\t\t {mlp_r2:.3f}\t\t {dt_r2:.3f}")
print(f"R-squared (R²):\t\t\t {rf_mae:.3f}\t {ada_mae:.3f}\t {mlp_mae:.3f}\t\t {dt_mae:.3f}")
print(f"Mean Absolute Error (MAE):\t {rf_mae:.3f}\t {ada_rmse:.3f}\t {mlp_rmse:.3f}\t {dt_rmse:.3f}")
print(f"Root Mean Squared Error (RMSE):\t {rf_rmse:.3f} {ada_mse:.3f}\t {mlp_mse:.3f}\t {dt_mse:.3f}")

In [ ]:
""" from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RepeatedStratifiedKFold

cv = RepeatedStratifiedKFold(n_splits=2, n_repeats=3, random_state=19)
rfc = RandomForestRegressor()

param_grid = [{'n_estimators': [100, 300, 500],
               'min_samples_split': [2, 5, 25],
               'min_samples_leaf': [3, 4, 5, 20],
               'max_depth': [5, 35, 50]}]

clf = GridSearchCV(rfc, param_grid = param_grid, n_jobs=-1, scoring='roc_auc', verbose=1)

clf.fit(X_train, y_train)
best_model = clf.best_estimator_ """

In [ ]:
""" results_df = pd.DataFrame(clf.cv_results_)

results_df = results_df.sort_values(by=["rank_test_score"])
results_df = results_df.set_index(
    results_df["params"].apply(lambda x: "_".join(str(val) for val in x.values()))
).rename_axis("kernel")
#print(results_df.columns)
results_df = results_df[["params", "rank_test_score", "mean_test_score", "std_test_score"]]

print(results_df.head(3)) """